<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Tools/blob/main/notebooks/GBIF_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget https://api.gbif.org/v1/occurrence/download/request/0417084-210914110416597.zip

--2022-08-05 20:35:54--  https://api.gbif.org/v1/occurrence/download/request/0417084-210914110416597.zip
Resolving api.gbif.org (api.gbif.org)... 130.225.43.2
Connecting to api.gbif.org (api.gbif.org)|130.225.43.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://occurrence-download.gbif.org/occurrence/download/request/0417084-210914110416597.zip [following]
--2022-08-05 20:35:54--  https://occurrence-download.gbif.org/occurrence/download/request/0417084-210914110416597.zip
Resolving occurrence-download.gbif.org (occurrence-download.gbif.org)... 130.225.43.36
Connecting to occurrence-download.gbif.org (occurrence-download.gbif.org)|130.225.43.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47415891 (45M) [application/octet-stream]
Saving to: ‘0417084-210914110416597.zip’

0417084-21091411041 100%[===================>]  45.22M  15.4MB/s    in 2.9s    

2022-08-05 20:35:58 (15.4 MB/s) - ‘0417084-210914110416597.zip’ saved 

In [2]:
%%time
! unzip -q 0417084-210914110416597.zip -d /content/

CPU times: user 23.8 ms, sys: 3.79 ms, total: 27.6 ms
Wall time: 2.32 s


In [4]:
import pandas as pd

occurrence_df = pd.read_csv("occurrence.txt", delimiter="\t", encoding="latin-1", on_bad_lines="skip")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,9,26,35,46,47,54,56,58,60,61,62,64,65,66,68,71,75,76,77,79,82,84,85,86,87,88,89,90,91,92,96,98,99,101,103,109,111,115,116,117,118,119,120,121,122,123,127,129,135,136,142,143,144,145,147,148,149,150,151,156,165,170,171,172,173,175,177,178,179,180,183,185,186,187,189,190,193,194,195,204,205,210,211,212,214,215,220,221,222,223,227,228,229,230,233,242,246) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
name_list = occurrence_df.columns
occurrence_df = pd.read_csv("occurrence.txt", delimiter="\t", encoding="latin-1", names=name_list)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,60,61,62,64,65,66,68,69,71,72,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,107,108,109,111,112,113,114,115,116,117,118,119,120,121,122,123,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,175,176,177,178,179,180,181,182,183,184,185,186,187,189,190,191,192,193,194,195,201,204,205,207,208,210,211,212,214,215,219,220,221,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,242,246,247,248,249,250,251,252,253,254,255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.use

In [6]:
occurrence_df = pd.DataFrame(occurrence_df.values[1:], columns=name_list)

In [7]:
multimedia_df = pd.read_csv("multimedia.txt", delimiter="\t", encoding="latin-1")

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [9]:
id_to_sp = {}
for idx, row in occurrence_df.iterrows():
  id_to_sp[row["gbifID"]] = row["acceptedScientificName"]

In [ ]:
import os
from tqdm.notebook import tqdm
from urllib import request
import time
from joblib import Parallel, delayed

rand_df = multimedia_df.sample(frac=1).reset_index(drop=True)
def get_ims(idx, row):
# for idx, row in tqdm(rand_df.iterrows(), total=len(rand_df)):
  id = row["gbifID"]
  try:
    sp = id_to_sp[id]
  except:
    return
  im_url = row["identifier"]
  

  try:
    filename = request.urlopen(request.Request(im_url)).url
  except:
    return
  bn = os.path.basename(filename)
  path = f"images/{sp}"

  if not os.path.isdir(path):
    os.makedirs(path)

  try:
    request.urlretrieve(im_url, f"{path}/{bn}")
  except:
    pass

  return

Parallel(n_jobs=8)(delayed(get_ims)(idx, row) for idx, row in tqdm(rand_df.iterrows(), total=len(rand_df)))

  0%|          | 0/122925 [00:00<?, ?it/s]